Pranav Kartha

8/13/2020
# INFO 370 2020 S - PS 8 - Validation  

## Background 

#### COMPAS Algorithm

COMPAS (Correctional Offender Management Profiling for Alternative Sanctions) is a commercial risk assessment tool that attempts to estimate a criminal defendent's recidivism (when a criminal reoffends, i.e. commits another crime). COMPAS is reportedly one of the most widely used tools of its kind in the US. It is often used in the US criminal justice system to inform sentencing guidelines by judges, although specific rules and regulations vary. 

In 2016, [ProPublica](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) published an investigative report arguing that racial bias was evident in the COMPAS algorithm. ProPublica had constructed a dataset from Florida public records, and used categorical logistic regressions and confusion matrices in its analysis. COMPAS's owners disputed this analysis, and [other academics](https://www.washingtonpost.com/news/monkey-cage/wp/2016/10/17/can-an-algorithm-be-racist-our-analysis-is-more-cautious-than-propublicas/) noted that for people with the same COMPAS score, but different races, the recidivism rates are effectively the same. 

The COMPAS algorithm is proprietary and not public. We know it includes 137 features, and deliberately excludes race. However, [another study](https://advances.sciencemag.org/content/4/1/eaao5580) showed that a logistic regression with only 7 of those features was equally accurate! 

Note: Links are optional readings, but can inform analysis/write up!

#### Dataset

The dataset you will be working with is based off ProPublica's [dataset](https://www.propublica.org/article/how-we-analyzed-the-compas-recidivism-algorithm), compiled from public records in Florida. However, it has been cleaned up for simplicity. You will only use a subset of the variables in the dataset for this exercise:

* c_charge_degree : Classifier for an individual's crime - F for felony, M for misdemeanor
* race : Classifier for the recorded race of each individual in this dataset. We will only be looking at 'Caucasian', and 'African-American' here.
* age_cat : Classifies individuals as under 25, between 25 and 45, and older than 45
* sex : Classifier for the recorded sex of each individual in this dataset. Male or female.
* priors_count: Numeric, the number of previous crimes the individual has committed.
* score_text: COMPAS classification of each individual's risk of recidivism (Low, Medium, or High)
* two_year_recid: Binary variable, 1 if the individual recidivated within 2 years, 0 otherwise.

#### Instructions

Your goals will be two-fold: 

1. (50 points)  Use confusion matrices to evaluate how well the COMPAS scores predict actual recidivism in two years, and investigate allegations of racial bias in the program.

2. (50 points) Demonstrate the issues and risk in overfitting. Create your own logistic regression model to predict recidivism, interpret the results for various categories. Then demonstrate issues with overfitting and spurious variables. 

## 1. Confusion Matrices

**a. Import the COMPAS data, and perform a basic data analysis. When satisfied, create a new dummy variable based off of COMPAS' risk score (score_text),  which indicates if an individual was classified as low risk or not.** 

* What is the recidivism rate for low-risk and high-risk individuals? 
* What about for low-risk African-American and low-risk Caucasian?

In [1]:
import pandas as pd
import numpy as np
import statistics as stat
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import copy
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [2]:
compas = pd.read_csv("compas-score-two-years-reduced-370.csv.bz2", usecols = ["c_charge_degree", "race", "age_cat", "sex", "priors_count", "score_text", "two_year_recid"] )



csize = compas.size
cnull = compas.isnull().sum().sum()

print("There are " + str(csize) + " entries in the dataset.")
print("There are " + str(cnull) + " null entries in the dataset.")  


There are 43204 entries in the dataset.
There are 0 null entries in the dataset.


In [3]:
scoredummy = []
for i in compas.score_text:
    if i == "Low":
        scoredummy.append(0)
    if i == "Medium" or i == "High":
        scoredummy.append(1)

        
compas.score_text = scoredummy



compas = compas[compas['race'].isin(["African-American","Caucasian"])] 

csize = compas.size
print("There are " + str(csize) + " entries in the dataset after dropping races other thatn Caucasian and African American.") 
print(compas.head(10)) 
print(compas.shape) 

lowrisk = compas[compas["score_text"].eq(0)]
hirisk = compas[compas["score_text"].eq(1)]

lowriskrecid = lowrisk[lowrisk["two_year_recid"].eq(1)]
hiriskrecid = hirisk[hirisk["two_year_recid"].eq(1)]


lowrecidrate = len(lowriskrecid)/len(lowrisk)
hirecidrate = len(hiriskrecid)/len(hirisk)

print("The recidivism rate for low risk individuals is " + str(lowrecidrate))
print("The recidivism rate for high risk individuals is " + str(hirecidrate))

lowriskaa = lowrisk[lowrisk["race"] == ("African-American")]
lowriskcauc = lowrisk[lowrisk["race"] == ("Caucasian")]

lowriskaarecid = lowriskaa[lowriskaa["two_year_recid"].eq(1)]
lowriskcaucrecid = lowriskcauc[lowrisk["two_year_recid"].eq(1)]


lowaarecidrate = len(lowriskaarecid)/len(lowriskaa)
lowcacerecidrate = len(lowriskcaucrecid)/len(lowriskcauc)

print("The recidivism rate for low risk African Americans is " + str(lowaarecidrate))
print("The recidivism rate for low risk Caucasians is " + str(lowcacerecidrate))



There are 36946 entries in the dataset after dropping races other thatn Caucasian and African American.
   c_charge_degree              race          age_cat  score_text     sex  \
1                F  African-American          25 - 45           0    Male   
2                F  African-American     Less than 25           0    Male   
4                F         Caucasian          25 - 45           1    Male   
6                M         Caucasian          25 - 45           0  Female   
7                F         Caucasian          25 - 45           0    Male   
8                M  African-American     Less than 25           1    Male   
9                M         Caucasian          25 - 45           0  Female   
10               F  African-American          25 - 45           0    Male   
11               F         Caucasian  Greater than 45           0  Female   
12               F  African-American          25 - 45           0    Male   

    priors_count  two_year_recid  
1            

C:\Users\prana\anaconda3\lib\site-packages\ipykernel_launcher.py:37: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


**b.  Now create a confusion matrix comparing COMPAS predictions for recidivism (is/is not low risk) and the actual two-year recidivism and interpret the results.**

NOTE: Do not just output a confusion matrix with accompanying text 'accuracy = x%', 'precision = y%'. Interpret your results such as 'z% of recidivists were falsly classified as low-risk, COMPAS accurately classified N% of individuals, etc.


In [4]:


riskmatrix = confusion_matrix(compas.two_year_recid, compas.score_text)



In [5]:
accuracy = accuracy_score(compas.two_year_recid, compas.score_text)
precision = precision_score(compas.two_year_recid, compas.score_text)
recall = recall_score(compas.two_year_recid, compas.score_text)
f1 = f1_score(compas.two_year_recid, compas.score_text)
 


print(str(accuracy* 100)+ "% of recidivists are classified correctly by COMPAS.")
print("Precision gives the postive predictive value. That is, given that a prediction to recidivate was made, the percentage of people who did actually recidivate among the people predicted to recidivate  was :" + str(precision* 100))
print("Recall is the percentage who did recidivate in accordance with the prediction with respect to all those who did recidivate, whether predicted or not.  The recall is " + str(recall* 100))

65.82038651004169% of recidivists are classified correctly by COMPAS.
Precision gives the postive predictive value. That is, given that a prediction to recidivate was made, the percentage of people who did actually recidivate among the people predicted to recidivate  was :63.445544554455445
Recall is the percentage who did recidivate in accordance with the prediction with respect to all those who did recidivate, whether predicted or not.  The recall is 64.51872734595247


**c. Note the accuracy of the COMPAS classification, and also how its errors were distributed. Would you feel comfortable having a judge use COMPAS to inform your sentencing guidelines? At what point would the error/misclassification risk be acceptable for you?**

Note: Given that judges are also not perfect, I will not consider 'zero error' an acceptable answer if you do not also consider human error in your answer.

The values calculated in part b indicate that there is a considerable number of recidivists classified incorrectly: all the measures calculated showed a 35 to 40% error rate. Due to this, I don't think COMPAS currently is a fair method of deciding sentencing guidelines. To become reliable, we need a much smaller error rate: 2-5% would be acceptable.

**d. Now, you will repeat your confusion matrix calculation and analysis from 1b. But this time for subsets of the population, first for only African-American individuals, and then for Caucasians.** 

* How accurate is the COMPAS classification for African-American individuals? For Caucasians?
* What are the false positive rates? (FP / (FP + TN)) 
* The false negative rates? (FN / (FN + TN)) 


In [6]:
compasaa = compas[compas.race == "African-American"]
compascauc = compas[compas.race == "Caucasian"]



riskmatrixaa = confusion_matrix(compasaa.two_year_recid, compasaa.score_text)

accuracyaa = accuracy_score(compasaa.two_year_recid, compasaa.score_text)
precisionaa = precision_score(compasaa.two_year_recid, compasaa.score_text)
recallaa = recall_score(compasaa.two_year_recid, compasaa.score_text)

riskmatrixaa = confusion_matrix(compasaa.two_year_recid, compasaa.score_text)
(tnaa, fpaa, fnaa, tpaa) = confusion_matrix(compasaa.two_year_recid, compasaa.score_text).ravel()

print(str(accuracyaa* 100)+ "% of African American recidivists are classified correctly.")
print("The precision for African American recidivists is " + str(precisionaa* 100))
print("The recall for African American recidivists" + str(recallaa* 100))



riskmatrixcauc = confusion_matrix(compascauc.two_year_recid, compascauc.score_text)
(tnc, fpc, fnc, tpc) = confusion_matrix(compascauc.two_year_recid, compascauc.score_text).ravel() 

accuracycauc = accuracy_score(compascauc.two_year_recid, compascauc.score_text)
precisioncauc = precision_score(compascauc.two_year_recid, compascauc.score_text)
recallcauc = recall_score(compascauc.two_year_recid, compascauc.score_text)



print(str(accuracycauc* 100)+ "% of Caucasian recidivists are classified correctly.")
print("The precision for Caucasian recidivists is " + str(precisioncauc* 100))
print(" The recall for Caucasian recidivists are " +str(recallcauc* 100))



print(str(fpaa/(fpaa+tnaa)) + " was the false positive rate for African Americans.")
print(str(fpc/(fpc+tnc)) + " was the false positive rate for Caucasians.") 

aafalseneg = riskmatrixaa[1,0]
aatrueneg = riskmatrixaa[0,0]
caucfalseneg = riskmatrixcauc[1,0]
cauctrueneg = riskmatrixcauc[0,0]

print("The false negative rate for African Americans is " + str(aafalseneg/(aafalseneg+ aatrueneg)))
print("The false negative rate for Caucasians is " + str(caucfalseneg/(caucfalseneg+ cauctrueneg)))


64.91338582677166% of African American recidivists are classified correctly.
The precision for African American recidivists is 64.95352651722253
The recall for African American recidivists71.52317880794702
67.18972895863052% of Caucasian recidivists are classified correctly.
The precision for Caucasian recidivists is 59.48275862068966
 The recall for Caucasian recidivists are 50.36496350364964
0.4233817701453104 was the false positive rate for African Americans.
0.22014051522248243 was the false positive rate for Caucasians.
The false negative rate for African Americans is 0.3514115898959881
The false negative rate for Caucasians is 0.2899786780383795


**e. If you've done this correctly, you will find that COMPAS's true negative and true positive percentages are similar for African-American and Caucasian individuals, but that false positive rates and false negative rates are different. Look again at the overal recidivism rates in the dataset for Black and White individuals. In your opinion, is the COMPAS algorithm 'fair'? Justify your answer.**


Hint: This is not a trick question. If you read the first two recommended readings - you will find that people disagree, generally by how you define fairness. Your answer will not be graded on which side you take, but on your justification.

The calculations done in part d show that Caucasians have a 20% higher false positive rate than African Americans, and a 6% lower false negative rate than African Americans. This, along with the fact that Caucasians have a higher accuracy ranking than African Americans. shows a bias against African Americans in the COMPAS algorithm.  Due to this, and that the model seems to have a high margin of error regardless of ethhnicity(see part d), I don't view the algorithm as fair.

## 2. Overfitting

Now we are going to explore issues and risks with overfitting data. 

In this section we expect you to use `sklearn` library.  It is a very convenient library in many ways, but inconvenient in other ways.

## a. First, create a new dataframe with only two_year_recid, charge_degree, age_cat, sex, and priors_count. 

Then, run a logistic regression with the data from this new dataframe that tries to explain the two year recidivism rate based off of all remaining variables (charge_degree, age_cat, sex, and priors_count). All variables except prior_count should be categorical.

* What is the accuracy of this model (computed on training data)? 
* What does it predict for a 22-yr old man with no prior charges arrested on a misdemeanor charge ?
* What about for a 65 yr-old woman with no priors charged with a felony? 

Hints:

* Hint 1: use `pd.get_dummies` to create categorical variables
* Hint 2: remember to drop the first (the reference category) when creating dummies!
* Hint 3: your design matrix _X_ should contain 5 columns (excluding intercept)
* Hint 4: the predicted probability for the 22yr man should be 0.5233 (but it may differ if you choose a different `C`)

In [7]:
charge_degree = pd.get_dummies(compas.c_charge_degree, drop_first = True)
age_cat = pd.get_dummies(compas.age_cat, drop_first = True)
sex = pd.get_dummies(compas.sex, drop_first = True)



Xyr = pd.concat((sex, age_cat, charge_degree,compas.priors_count,compas.two_year_recid), axis=1) 
X = pd.concat((sex, age_cat, charge_degree,compas.priors_count), axis=1) 



m = LogisticRegression(solver="lbfgs", max_iter = 1000)
first = m.fit(X, compas.two_year_recid)
print("The accuracy of this model is " + str(m.score(X,compas.two_year_recid)))



youngman = m.predict_proba(np.column_stack((1,0,1,1,0)))
print("The probability that a 22 year old man with no prior charges, arrested on a misdemeanor, commits another crime is " + str(youngman[0,1]))

oldwoman = m.predict_proba(np.column_stack((0,1,0,0,0)))

print("The probability that a 65 year old woman with no prior charges, arrested on a felony, commits another crime is " + str(oldwoman[0,1]))



The accuracy of this model is 0.665403561955286
The probability that a 22 year old man with no prior charges, arrested on a misdemeanor, commits another crime is 0.4992151261465721
The probability that a 65 year old woman with no prior charges, arrested on a felony, commits another crime is 0.1655215515633825


### b. Now randomly sample 50 observations from your dataframe, and run the same regression you did in 2a.

* What is the accuracy now (on training data)?
* Is it the same or different than your previous model?
* How about the coefficients? 
* Why do you think this is?

In [8]:
sample = Xyr.sample(50)
recids = sample.two_year_recid
samplenorecids = sample.drop(columns=['two_year_recid'])
m1 = LogisticRegression(solver="lbfgs", max_iter = 1000) 
samplefirst = m1.fit(samplenorecids, recids)


print("The accuracy of this model is " + str(m1.score(samplenorecids,recids)))
print(first.coef_)
print(samplefirst.coef_)
print("The sample had a higher accuracy score than the whole data, with some differences appearing in the coefficients.  A reason for this could be that as the data set increases in size, there is a greater proportion of outlier data points, resulting in a weaker trend.")

The accuracy of this model is 0.72
[[ 0.35386789 -0.71419927  0.73981949 -0.19332065  0.16496787]]
[[ 1.41944282 -0.45954367  0.9082655   0.0204523   0.09874088]]
The sample had a higher accuracy score than the whole data, with some differences appearing in the coefficients.  A reason for this could be that as the data set increases in size, there is a greater proportion of outlier data points, resulting in a weaker trend.


### c. Now we will add fake data to the sample we created in 2b. 

Create 25 variables of random data and add it to the dataframe you created in 2b. Repeate 2b with this new, random-extended, dataframe. Which model has a higher accuracy?

Hint: your design matrix _X_ should contain 30 columns (excluding intercept)

In [9]:
sample2 = sample





for j in range(25):
    randomvar = []
    for i in range(50):
        randomvar.append(np.random.normal(0,5000,size = 1))
    colname = "random" + str(j)
    sample2[colname] = randomvar
    


recids2 = sample2.two_year_recid
sample2norecids = sample2.drop(columns=['two_year_recid'])




m2 = LogisticRegression(solver="lbfgs", max_iter = 100000)
sample2_ = m2.fit(sample2norecids, recids2)
print("The accuracy of this model is " + str(m2.score(sample2norecids,recids2)))


The accuracy of this model is 1.0


### d. Why do we get a better accuracy when we add random data to the model? 

Can we use this trick to predict the stock market prices and get rich (and I mean seriosly rich)? Why wouldn't that work?


Adding random data to the model creates a regression that is overfitted, or too fined tuned to the data points that we have used to create the regression, and cannot be accurately applied to new data. Thus, using this method would not provide any accurate predictions for any real world scenarios.

### e. Now 10-fold cross validate models 2b (sample w/o random variables) and 2c (sample w/random variables)

Use accuracy as the CV score

In [10]:
scoresb = cross_val_score(m1, samplenorecids,recids, cv=10)
print("CV(b) results:", scoresb)
print("CV(b) score:", np.mean(scoresb))


scoresc = cross_val_score(m2, sample2norecids,recids2, cv=10)
print("CV(c) results:", scoresc)
print("CV(c) score:", np.mean(scoresc))

CV(b) results: [0.6 0.6 0.8 0.4 0.8 0.8 0.6 1.  0.4 0.8]
CV(b) score: 0.68
CV(c) results: [0.2 0.6 0.8 0.8 0.4 0.8 0.6 0.4 0.6 0.4]
CV(c) score: 0.56


### f. Explain what did you get

* Why does cross-validation disagree with your results in 2b and 2c?

Cross validation results in the model in part b being more accurate than the model in part c. When the model isn't trained with the garbage data, unlike in part c, it is more applicable to new data. In part c, the model was overfitted, meaning it was not applicable to multiple sets of data.  Since cross validation involves testing a model against different data sets(through using different testing and training sets), it would give a lower score to an overfitted model, which in this case, it has. 

## 3. Extra Credit: Implement Cross-Validation (1ec point)


Here you will implement a cross-validation function yourself without any dedicated libraries (you are still welcome to use numpy, pandas, and other basic libraries).  Consult James et al (2015) Introduction to Statistical Learning with R, Section 5.1 for more
information. 

We choose accuracy as the goodness measure.

* why do we choose accuracy instead of RMSE?

RMSE is generally used on continuous sets of data while accuracy is used in binary sets of data. The variable we are using is binary, so accuracy would be more applicable here.

Write the cross-validation function.  The function should take three inputs:
1. the estimated model (you may use either sklearn or smf)
2. the test data design matrix $X$
3. and the test labels $y$.

You may add other inputs if you consider it useful, for instance how many folds, controls for print verbosity, etc.

The function should broadly do the following:
1. put your data into random order
2. split these into $k$ chunks
3. select a chunk for testing and the others for training
4. train your model on the training chunks
5. compute accuracy on the training chunk
6. return mean accuracy over all these $k$ trials.

Finally, repeat question 2e with your own cross-validation function and show that you get similar results!

In [11]:
def crossval(model, x, y, num_chunks):
    

    xcopy = copy.deepcopy(x)
    xcopy["yvals"] = copy.deepcopy(y)
    xrandom = xcopy.sample(frac = 1)
    
    chunks = []
    accuracys = []
    place = 0
    chunk_size = int(len(xrandom)/num_chunks)
    
    
    #splitting the data into num_chunk chunks, each of size chunk_size
    for i in range(num_chunks):
        chunk = []
        for j in range(chunk_size):
            chunk.append(xrandom.iloc[place])
            place += 1
        chunks.append(pd.DataFrame(chunk))
    
    for i in range(len(chunks)):
         #Choose the testing chunk as ith element in chunks and the training chunks as the other elements 
        testing = chunks[i]
        testingys = testing.yvals
        testing =testing.drop(columns = ["yvals"])
    
        training = chunks[i]  
        training = training.iloc[0:0]
       
         #populating data frame for training set data
        for j in range(len(chunks)):
            if(j != i):
                 training = training.append(chunks[j])
        
        trainingys = training.yvals  
        training = training.drop(columns = ["yvals"])
        m1 = copy.deepcopy(model) 
        m = m1.fit(training, trainingys)
 
        accuracy = m.score(testing, testingys)
        accuracys.append(accuracy)
    return stat.mean(accuracys)
        
    
m = LogisticRegression(solver="lbfgs", max_iter = 10000)

print("The accuracy of the model in part b is " + str(crossval(m,samplenorecids, recids,10)) + " similar to the values in obtained in the in built function.")
print("The accuracy of the model in part c is " + str(crossval(m,sample2norecids, recids2,10)) + " similar to the values in obtained in the in built function.")  
            
        

The accuracy of the model in part b is 0.68 similar to the values in obtained in the in built function.
The accuracy of the model in part c is 0.62 similar to the values in obtained in the in built function.
